# SOME BACKGROUND

~ STEP 1 :- First download the latest Python Veriosn 
            https://www.python.org/downloads/

Please Install the downloaded file on your computer using custom Installation settings. 

~ STEP 2 :- After the installation is done , Open  , In MAC - Terminal or WIN - Terminal , and type "Python3 --version"
 for MAC or "Python --version" for WIN and press Enter 

 If , the Terminal , returns with the verion of Python like "Python 3.11.1" , Installation has been succesful , and then we can proceed further , if not , please check if STEP 1 has been accomplished

~STEP 3 :- After we are done  with the STEP 2 , I am using the Jupytr Lab for the tutorials. , type in terminal "pip install jupyter lab" , this will install the latest verison of Jupyter Lab on your system , automatically you will be redirected to the browser 

~ STEP 4 :- To check , if the installation of Jupyter lab has been successful , type "jupyter lab --version" on Terminal. If , the Terminal , returns with the verion of Jupyter Lab like "4.1.2" , Installation has been succesful , and then we can proceed further , if not , please check if STEP 3 has been accomplished

~ STEP 5 :- A library is a collection of code that makes everyday tasks more efficient. For my tutorial i would recommend Installing pandas , numpy - For data analysis , seaborn , matplotlib - For Visualisation.  

Also , i would be Installing a library called "peakutils" , this woulbe be useful for recogisation of Peaks from the Spectra. https://peakutils.readthedocs.io/en/latest/


~ STEP 6 :- For installing the library , go to terminal , and type "pip install pandas" and after installing . Similarly , do this for numpy , seaborn , matplotlin and PeakUtils

~ STEP 7 :- Now , to open the Jupyter Lab seession in the browser , go to terminal and type "jupyter lab" , you will be automatically redirected to the browser in a jupyter lab session. This is where you will start programming . 

NOTE :- To exit the session , press "Ctrl + C" on the termianal , this will end the running session on your system

~ STEP 8 :- In jupyter lab , your lines of codes or chunk of lines of codes  are called as "Cell" , If the "Cell are of type Markdown - Used for Comments / Information " and If the "Cells are of type Code - Used for Programming" 

You can get more in fo here -  https://www.datacamp.com/tutorial/installing-jupyter-notebook


# Importing the Useful librararies. 

Line [1] is completely optional , This is used to Hide the the Warnings in the Output cell. It's recommended to skip the line[1] , and proceed with the code from line[2].

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import statistics
###############################################
from peakutils import indexes
from peakutils import baseline
from scipy.signal import find_peaks as fp
from scipy.signal import savgol_filter 
###############################################
from bokeh.plotting import figure , show

from bokeh.io import output_notebook
output_notebook()


Loading BokehJS ...

# LOADING THE SPECTRA FILE WHICH WE HAVE

In [3]:
file_path = 'Spectra/CR240LA/1780.csv'

Sample_Spectra = pd.read_csv(file_path)
Sample_Spectra = Sample_Spectra.rename({'intensity':'Intensities'},axis=1)


Sample_Spectra_Plot = figure(title = 'Raw Sample Spectra' , x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')
Sample_Spectra_Plot.line(Sample_Spectra.wavelength,Sample_Spectra.Intensities , line_width = 2, color ="green" )
Sample_Spectra_Plot.width = 1200
Sample_Spectra_Plot.height = 600

print (Sample_Spectra)
show (Sample_Spectra_Plot)


       wavelength  Intensities
0      180.000000          0.0
1      180.033333          0.0
2      180.066667          0.0
3      180.100000          0.0
4      180.133333          0.0
...           ...          ...
23426  960.866667          0.0
23427  960.900000          0.0
23428  960.933333          0.0
23429  960.966667          0.0
23430  961.000000          0.0

[23431 rows x 2 columns]


Now , as we see in the figure avove , the spectra have lots of lines , which are very specific to the element present in the Sample . 

Each element has an specific , wavelength associated with it.

Now , i have visited the NIST LIBS database , and download the spectra for all the element in the Periodic Table. The File are updated with that of NIST database as of 17 April 2024.

# CHOOSING THE ELEMENT OF INTEREST

In [4]:

element = 'Al' # ENTER YOUR ELEMENT OF CHOICE
Reference_Spectra = pd.read_csv(r"NIST/" + element +".csv" , encoding = 'windows - 1252' , usecols=(0,1))
print(Reference_Spectra)

Reference_Spectra_Plot = figure(title = 'NIST Reference Library ', x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')
Reference_Spectra_Plot.line(Reference_Spectra.Wavelength,Reference_Spectra.Spectra , line_width = 2, color ="blue" )
Reference_Spectra_Plot.width = 1200
Reference_Spectra_Plot.height = 600

show(Reference_Spectra_Plot)

      Wavelength       Spectra
0         170.91  6.110000e-07
1         170.96  1.830000e-05
2         171.00  2.460000e-04
3         171.04  2.960000e-03
4         171.08  3.210000e-02
...          ...           ...
7618      966.21  1.050000e-16
7619      966.26  5.710000e-17
7620      966.30  3.490000e-17
7621      966.34  2.130000e-17
7622      966.38  1.290000e-17

[7623 rows x 2 columns]


# APPROACH FOR PEAK DETECTION IN THE NIST ELEMENT SPECTRA

Now, uptil now , we have selected the Material Sample ,which we want to Interact with and the data is stored in variable "Sample_Spectra"
Also , we have choosen the element , which we want to detect in this 'Sample_Spectra' , and the spectral information for that element is stored in variable "Reference_Spectra" 

Now , from this 'Reference_Spectra' , Let's try to get all the Peaks this Spectra has

I have used a library called as Peak_Utils 

https://peakutils.readthedocs.io/en/latest/reference.html

The detail of the function is given below. 

---------------------------------------------------------------------------------------------------------------------------

NOTE :- Now if we keep the minimum_distance_1 = 0 and threshold_1 = 0 , we get all the peaks in the spectra , Tuning of these parameter are most important as per your signal.

Setting the value in parameter 'minimum_distance_1' , will allows to skip the peaks , which are very close to each other. 
Setting the value in parameter 'threshold_1', will allows to skip the peaks with low intensity and also seperate the Peaks , with could be the noise signal. 

So , its important to choose the value's as per your convenience 

----------------------------------------------------------------------------------------------------------------------------

In [5]:
minimum_distance_1 = 0 
threshold_1 = 0.01 #looking for a value above 0.01% , so that Peak with very low Intensities can be ignored 

Reference_Peak = indexes(Reference_Spectra.Spectra , min_dist=minimum_distance_1 , thres=threshold_1) 

Selected_Reference_Spectra = Reference_Spectra.iloc[Reference_Peak] #from the indices , lets try to find the Wavelength and Spectra. 

Reference_Spectra_Peaks = figure(title = 'Peaks in NIST Spectra ', x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')
Reference_Spectra_Peaks.line(Reference_Spectra.Wavelength,Reference_Spectra.Spectra , line_width = 2, color ="blue" )
Reference_Spectra_Peaks.circle(Selected_Reference_Spectra.Wavelength , Selected_Reference_Spectra.Spectra , color = 'red' , size = 8)

Reference_Spectra_Peaks.width = 1200
Reference_Spectra_Peaks.height = 600

print(Selected_Reference_Spectra)
show(Reference_Spectra_Peaks)



      Wavelength       Spectra
28        172.10  7.880000e+09
37        172.49  1.170000e+10
119       176.40  8.450000e+09
299       185.79  1.450000e+09
309       186.22  2.190000e+09
428       193.11  1.350000e+08
551       199.06  1.010000e+09
799       212.32  1.670000e+08
814       212.96  1.300000e+08
826       213.47  2.550000e+08
852       214.58  1.800000e+08
864       215.09  3.540000e+08
906       216.87  2.700000e+08
919       217.43  5.280000e+08
991       220.49  4.380000e+08
1003      221.00  8.320000e+08
1089      226.35  7.660000e+08
1102      226.91  1.330000e+09
1300      236.73  9.360000e+08
1314      237.32  1.880000e+09
1468      256.80  3.030000e+08
1485      257.52  9.380000e+08
1610      265.26  1.220000e+08
1628      266.02  2.420000e+08
1787      281.62  1.630000e+08
1871      308.20  1.430000e+09
1896      309.26  2.940000e+09
2493      394.38  8.960000e+08
2535      396.16  1.760000e+09


# Wavelengths for Detected Peak(red circles above)

In [6]:
Reference_Spectra_List = []

for iteam in Selected_Reference_Spectra.Wavelength:
    iteam = float(iteam)
    if iteam > 187.0:        #Only those iteam will be selected , whose wavelength would be greater than 188 nm ,  as our Handheld LIBS range starts from 188nm
        Reference_Spectra_List.append(iteam)
    
print(Reference_Spectra_List) #These are the wavelengths of Peaks , which we are interested in the NIST Spectra
Length_Reference_Spectra_List = (len(Reference_Spectra_List))   #Used for Calculating Percentage Detection

[193.11, 199.06, 212.32, 212.96, 213.47, 214.58, 215.09, 216.87, 217.43, 220.49, 221.0, 226.35, 226.91, 236.73, 237.32, 256.8, 257.52, 265.26, 266.02, 281.62, 308.2, 309.26, 394.38, 396.16]


# 0.0333 is the sprectral resolution of the Handheld LIBS Device

Now , for the two wavelengths from two different data source , to exactly match with each other , is going to be difficult because of the spectral resolution of wavelength - for these two data source are very different from each other.

Now we know that the Spectral Resolution of Handheld LIBS Device is 0.0333. 

Now , we have a list of all wavelengths , detected for Peaks in Reference Spectra , Now for each element lets try to create a min and max of each element in the list. 'Reference_Spectra_Min_Max' is list consisting of a sublist , the sublist have a min and max value or a range of element in the list.


Eg : - A peak of 254.8 in NIST Spectra , can be found at and between [254.7668 ~ 254.8332] in the Handheld LIBS Spectra , because the resolution of wavelength in NIST Spectra and Handheld LIBS Spectra is different. 


In [7]:
Reference_Spectra_Min_Max = []
Reference_Spectra_Min_Max = [[num - 0.0332, num + 0.0332] for num in Reference_Spectra_List]     
    
print(Reference_Spectra_Min_Max)

[[193.07680000000002, 193.1432], [199.0268, 199.0932], [212.2868, 212.3532], [212.92680000000001, 212.9932], [213.4368, 213.5032], [214.54680000000002, 214.6132], [215.0568, 215.1232], [216.8368, 216.9032], [217.3968, 217.4632], [220.45680000000002, 220.5232], [220.9668, 221.0332], [226.3168, 226.3832], [226.8768, 226.9432], [236.6968, 236.76319999999998], [237.2868, 237.3532], [256.7668, 256.83320000000003], [257.48679999999996, 257.5532], [265.22679999999997, 265.2932], [265.98679999999996, 266.0532], [281.5868, 281.6532], [308.16679999999997, 308.2332], [309.22679999999997, 309.2932], [394.3468, 394.4132], [396.1268, 396.19320000000005]]


# Pre-Processing the Spectra for Handheld LIBS to improve the accuracy of Peak Detection 

As we see in our raw spectra , there are lot of small peaks , which really aren't emission peaks , but are the noise in the signal. To tackle that Smoothing the Spectra is usually done. 

I am sing the Savitsky Golay , for smoothing my spectra. 

In [8]:
win_length =10
pol_order = 0 #Pol_order must be less than windows length

Filtered_Sample_Spectra = savgol_filter(Sample_Spectra.Intensities , window_length=win_length , polyorder=pol_order)

Smooth_Spectra = figure(title = 'Smoothed hLIBS Spectra', x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')
Smooth_Spectra.line(Sample_Spectra.wavelength,Sample_Spectra.Intensities , line_width = 3, color ="green" )
Smooth_Spectra.line(Sample_Spectra.wavelength,Filtered_Sample_Spectra, line_width = 1, color ="red" )


Smooth_Spectra.width = 1200
Smooth_Spectra.height = 600

show(Smooth_Spectra)


# Noise Estimation
As we see in the Handheld LIBS Spectra , It's difficult to find or detect , what is the noise and what is the signal.

But we have an information , that the range of Spectrometer in hLIBS starts from 190nm , So we can say that the signal detected uptil 190nm could be classified as noise.

In [9]:
noise_list = [] 
noise_start = []

noise_list = Sample_Spectra.loc[Sample_Spectra['wavelength'] < 190.0 , 'Intensities']

RMS_intensity_noise = max(noise_list) #np.sqrt(np.mean((noise_list)**2)) #This is the max Intensity of the noise signal
print(RMS_intensity_noise)  



63.80682334948245


In [10]:
#METHOD  - Scipy

Threshold = 0.1 #Noise Intensity + Threshold Value for signal filtering
height_2 = None
Peak_Prominenace = (RMS_intensity_noise + Threshold)  #Uses S/N ratio , as comapred to other peaks
minimum_distance_3 = None   #minimum distance between two peaks

Sample_Peak , _ = fp(Sample_Spectra.Intensities,height=height_2,prominence=Peak_Prominenace,distance=minimum_distance_3)
Selected_Sample_Spectra = Sample_Spectra.iloc[Sample_Peak] 

Sample_Spectra_Peaks = figure(title = 'Peaks in hLIBS Spectra', x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')
Sample_Spectra_Peaks.line(Sample_Spectra.wavelength,Sample_Spectra.Intensities , line_width = 2, color ="green" )
Sample_Spectra_Peaks.circle(Selected_Sample_Spectra.wavelength , Selected_Sample_Spectra.Intensities , color = 'red' , size = 8)

Sample_Spectra_Peaks.width = 1200
Sample_Spectra_Peaks.height = 600

print(Selected_Sample_Spectra)
show(Sample_Spectra_Peaks)

       wavelength  Intensities
393    193.100000  4800.257819
399    193.300000   290.321771
409    193.633333   232.969054
508    196.933333   202.020311
571    199.033333   845.727836
...           ...          ...
22272  922.400000   968.978144
22662  935.400000   141.325590
22756  938.533333   187.565790
22816  940.533333   160.843975
22977  945.900000   235.461752

[259 rows x 2 columns]


# Wavelengths for Detected Peak(red circles above)

In [11]:
Sample_Spectra_List = []

for iteam in Selected_Sample_Spectra.wavelength:
    iteam = float(iteam)
    Sample_Spectra_List.append(iteam)
    
print(Sample_Spectra_List)


[193.09999999999928, 193.29999999999924, 193.63333333333256, 196.9333333333324, 199.03333333333225, 202.59999999999872, 204.09999999999863, 204.89999999999856, 205.3333333333319, 206.23333333333184, 206.43333333333183, 207.0333333333318, 207.8999999999984, 208.6999999999984, 209.466666666665, 209.6999999999983, 209.9999999999983, 210.26666666666492, 210.4333333333316, 210.66666666666487, 212.4333333333315, 213.86666666666477, 220.1666666666644, 220.99999999999767, 221.63333333333097, 226.3333333333307, 226.89999999999733, 233.26666666666364, 233.7666666666636, 234.33333333333024, 234.79999999999688, 235.96666666666349, 236.6999999999968, 237.33333333333007, 238.1999999999967, 238.83333333333, 239.53333333332995, 239.8666666666633, 240.46666666666323, 241.0666666666632, 241.33333333332985, 243.46666666666303, 245.89999999999625, 247.26666666666284, 247.56666666666283, 247.83333333332948, 248.2999999999961, 248.7999999999961, 249.0333333333294, 249.2999999999961, 249.9666666666627, 250.1

# So far so now , i have two list with me 

1) Sample_Spectra_List = Containg wavelngths of all Peaks from data of Handheld LIBS device
2) Reference_Spectra_Min_Max = Containing a range of wavelength , for all the peak detected in the Reference Sample

   

In [12]:
print(Sample_Spectra_List)

[193.09999999999928, 193.29999999999924, 193.63333333333256, 196.9333333333324, 199.03333333333225, 202.59999999999872, 204.09999999999863, 204.89999999999856, 205.3333333333319, 206.23333333333184, 206.43333333333183, 207.0333333333318, 207.8999999999984, 208.6999999999984, 209.466666666665, 209.6999999999983, 209.9999999999983, 210.26666666666492, 210.4333333333316, 210.66666666666487, 212.4333333333315, 213.86666666666477, 220.1666666666644, 220.99999999999767, 221.63333333333097, 226.3333333333307, 226.89999999999733, 233.26666666666364, 233.7666666666636, 234.33333333333024, 234.79999999999688, 235.96666666666349, 236.6999999999968, 237.33333333333007, 238.1999999999967, 238.83333333333, 239.53333333332995, 239.8666666666633, 240.46666666666323, 241.0666666666632, 241.33333333332985, 243.46666666666303, 245.89999999999625, 247.26666666666284, 247.56666666666283, 247.83333333332948, 248.2999999999961, 248.7999999999961, 249.0333333333294, 249.2999999999961, 249.9666666666627, 250.1

In [13]:
print(Reference_Spectra_Min_Max)

[[193.07680000000002, 193.1432], [199.0268, 199.0932], [212.2868, 212.3532], [212.92680000000001, 212.9932], [213.4368, 213.5032], [214.54680000000002, 214.6132], [215.0568, 215.1232], [216.8368, 216.9032], [217.3968, 217.4632], [220.45680000000002, 220.5232], [220.9668, 221.0332], [226.3168, 226.3832], [226.8768, 226.9432], [236.6968, 236.76319999999998], [237.2868, 237.3532], [256.7668, 256.83320000000003], [257.48679999999996, 257.5532], [265.22679999999997, 265.2932], [265.98679999999996, 266.0532], [281.5868, 281.6532], [308.16679999999997, 308.2332], [309.22679999999997, 309.2932], [394.3468, 394.4132], [396.1268, 396.19320000000005]]


# Finally , My  idea is see , if each iteam in (Reference_Spectra_Min_Max) , matches with each element in (Sample_Spectra_List) , and then caluclate the exact matching Percentage 


In [14]:
plot_spectra_wavelength_list=[]
plot_reference_wavelength_list = []
count = 0
for cell in Reference_Spectra_Min_Max: 
    for iteam in Sample_Spectra_List:
        if iteam >= cell[0] and iteam <= cell[1]:
            count = count + 1 #for counting
            plot_spectra_wavelength_list.append(iteam)
            plot_reference_wavelength_list.append(cell[0] + 0.0333)

plot_spectra_intensities_list=[]
for iteam in plot_spectra_wavelength_list:
    Inte = Selected_Sample_Spectra.loc[Selected_Sample_Spectra['wavelength'] == iteam, 'Intensities'].iloc[0]
    plot_spectra_intensities_list.append(Inte)
#print(plot_spectra_intensities_list)

#print(plot_spectra_wavelength_list)

print('The total peaks detected were ' + str(count) + '\n' )

print('The following peaks , for Reference Sample was detected  :- '  )
print(plot_reference_wavelength_list )

print('\n')
print('The Sample Peaks detected from the sample were:-')
spectra_df = pd.DataFrame({'wavelength': plot_spectra_wavelength_list, 'intensities': plot_spectra_intensities_list})
print(spectra_df)  

The total peaks detected were 15

The following peaks , for Reference Sample was detected  :- 
[193.11010000000002, 199.0601, 221.0001, 226.3501, 226.9101, 236.7301, 237.3201, 257.52009999999996, 265.26009999999997, 266.02009999999996, 281.6201, 308.20009999999996, 309.26009999999997, 394.38009999999997, 396.1601]


The Sample Peaks detected from the sample were:-
    wavelength  intensities
0   193.100000  4800.257819
1   199.033333   845.727836
2   221.000000   334.752248
3   226.333333   411.756364
4   226.900000   519.170989
5   236.700000   773.255349
6   237.333333  1191.804515
7   257.500000  1329.747220
8   265.233333   294.192095
9   266.000000   406.524912
10  281.600000   525.976357
11  308.200000  2868.381800
12  309.266667  4147.559041
13  394.400000  6681.622164
14  396.166667  9718.572650


# RESULTS

In [15]:
percentage = (count / Length_Reference_Spectra_List) * 100
percentage

print (' Ananlysis of ' + element + ' was done , and the ' + str(count) +  ' number of spectral lines from the material sample ' + ' matched with the ' + str(Length_Reference_Spectra_List) + ' choosen Reference element of '+ element + ' therefore , we can say' ,  str(percentage) + '%' + ' matched as per the parameters set by the user')

 Ananlysis of Al was done , and the 15 number of spectral lines from the material sample  matched with the 24 choosen Reference element of Al therefore , we can say 62.5% matched as per the parameters set by the user


Find the Correponding Intensities , for the wavelengths available in "plot_spectra_wavelength_list" and also for the wavelengths available in "plot_refernce_intensities_list"

# Generate a Interactive Plot

In [16]:
p = figure(title = 'Peak Detection' , x_axis_label = 'Wavelength' , y_axis_label = 'Intensities')
p.line(Sample_Spectra.wavelength,Sample_Spectra.Intensities , line_width = 2, color ="red" )
p.circle(spectra_df.wavelength, spectra_df.intensities , color = 'green' , size = 8)

p.width = 1200
p.height = 600

show(p)

